# Functionality: visualization of functional connectivity

<ul>
    <li>visualize the ciCOH in mild and normal state</li>
</ul>

## Import Packages 

In [1]:
import os, sys
import numpy as np
import pandas as pd

import pickle

from igraph.drawing.text import TextDrawer
import cairo

In [2]:
# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)


# import animal_folder function in util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual

importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/graphVisual.ipynb


## exp subfolders & code_corresfolder

In [3]:
_, _, pipelinefolder, _ = exp_subfolders()

In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder = code_corresfolder(os.getcwd(), nb_name)

## global parameters

In [19]:
# threshold lowweight
lowweight = 0.1

# frequency
freq = [9, 11]
strfreq = 'freq' + str(freq[0]) + '_' + str(freq[1])


# eventphase
event = 'rest'

In [20]:
### text setup for brain areas ###
texts_area = dict()

texts_area['Caudate'] = [100, 30, 20]

texts_area['VPLo'] = [100, 140, 20]

texts_area['VLo'] = [100, 240, 20]

texts_area['VA'] = [100, 360, 20]

texts_area['MC'] = [100, 460, 20]

texts_area['SMA'] = [100, 560, 20]

## input setup

In [31]:
inputfolder_ciCOH = os.path.join(pipelinefolder, 'NHP_Pinky','1_dataAnaly', 'FCAnaly', 'Rest', 'm1_calcciCOH_rest')
filename_ciCOH = 'ciCOH_rest_freq9_11_mild_110917_bktdt4.npy'
#filename_ciCOH = 'ciCOH_rest_freq9_11_normal_041417_bktdt6.npy'
file_ciCOH = os.path.join(inputfolder_ciCOH, filename_ciCOH)

In [32]:
chnInf_folder = os.path.join(pipelinefolder, 'NHP_Pinky','1_dataAnaly', 'FCAnaly', 'Rest')

chnInf_file = os.path.join(chnInf_folder, 'chn_brainArea_simCoord_ThaSMA.csv')

## savefile setup

In [33]:
savefolder = corresfolder
savefile_prefix = 'ciCOH_' + event + '_' + strfreq + '_lowweight' + str(lowweight)

In [41]:
print(filename_ciCOH)

ciCOH_rest_freq9_11_mild_110917_bktdt4.npy


## extract Channel Information

In [34]:
### load channel information ###
df = pd.read_csv(chnInf_file, header = 0)


###  channel matchni and recording chni  ###
chnInf_matchni, chnInf_recchni = df['mat_chni'], df['recording_chni']

chnInf_brainarea = df['brainarea']

x, y = df['simulated_x'].to_numpy(), df['simulated_y'].to_numpy()
chnInf_coord = np.concatenate((np.expand_dims(x, axis = 1), np.expand_dims(y, axis = 1)), axis = 1)



### Remove nonused channels ###

# extract the indices in which the corresponding channels are not used
idx_mask = ~chnInf_brainarea.isnull()
idx_notused = [i for i, isnan in enumerate(idx_mask) if isnan == False]

chnInf_recchni = chnInf_recchni[idx_mask].to_numpy()

chnInf_brainarea = chnInf_brainarea[idx_mask].tolist()

chnInf_coord = chnInf_coord[idx_mask]

## extract ciCOH

In [35]:
# ciCOH_normal, ciCOH_mild: np.ndarray, shape = (nchns, nchns)
if 'mild' in file_ciCOH:
    ciCOH_mild = np.load(file_ciCOH)
if 'normal' in file_ciCOH:
    ciCOH_normal = np.load(file_ciCOH)

## ciCOH Visual

In [36]:
def ciCOH_visual_save(ciCOH, chn_area, chn_coord, str_recchnishow, lowweight, tobesavedfile, texts):
    """
        
        Args:
            
            ciCOH (np.sdarray): ciCOH matrix (nchns, nchns)
            
            chn_area (list): a list of string representing brain area for each channel, len(chn_area) = nchns
            
            chn_coord (numpy.ndarray): x, y coordinates for each channel, shape = (nchns, 2)
            
            chn_recchni(list): showed recording chni for each channel, shape = (nchns, )
            
            lowweight: the threshold lowweight, only weight>lowweight is treated as connection
            
            tobesavedfile: file to save the visualized figure
            
            texts:
            
        Output:
            the visualizaton of ciCOH is saved in tobesavedfile
            
    """
    
    weight = abs(ciCOH)

    # weight > lowweight
    weight[np.where(weight<lowweight)] =0
    
    graph = graphVisual.graph_create(weight, vsarea = chn_area, vscoord = chn_coord, vschni = str_recchnishow)

    # set graph visualization style
    visual_style = graphVisual.graph_style(graph)
    

    igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
    
    igplot.save(tobesavedfile)
    
    return igplot

### outer 

In [37]:
nchns = len(chnInf_recchni)

In [38]:
str_recchnishow = [''] *  nchns

In [39]:
"""
    normal condition
"""


# save file config
savefilename = savefile_prefix + '_normal' + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()


# visualize and save
igplot = ciCOH_visual_save(ciCOH_normal, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = str_recchnishow,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts)

igplot.show()

print("Figure saved to " + savefile)


Figure saved to /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/1_dataAnaly/FCAnaly/Rest/m2_visual_ciCOH_rest/ciCOH_rest_freq9_11_lowweight0.1_normal.png


In [ ]:
"""
    mild condition
"""

condition = 'mild'


# save file config
savefilename = savefile_prefix + '_' + condition + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()


# visualize and save
igplot = ciCOH_visual_save(ciCOH_mild, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = str_recchnishow,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts)

igplot.show()

print("Figure saved to " + savefile)